# Swap

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) and the Refinitiv Data Libraries.

##### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

#### Import the library and connect to the Refinitv Data Libraries

In [1]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import swap

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x1036202b0 {name='workspace'}>

### Pricing a Vanilla IRS

This sample price a swap defined leg by leg for a complete control of its term and conditions. The 'fields' requested are:
- the gross price in percent (DirtyPricePercent), which is set to null for the solving process
- the fixedRatePercent of the fixed leg
- the spreadBp over the index of the float leg
- the PV01 (PV01Bp,PV01AmountInDealCcy)
- the names of the zero-coupon curves used (DiscountCurveName,ForwardCurveName)
- the error information (ErrorCode,ErrorMessage)

Solving Workflow:
- if neither fixedRatePercent nor spreadBp are defined in the legs, spreadBp is considered null and fixedRatePercent is solved so that the DirtyPricePercent of both legs match
- if the spreadBp value is set, fixedRatePercent is solved so that the DirtyPricePercent of both legs match
- if fixedRatePercent value is set, spreadBp is solved so that the DirtyPricePercent of both legs match


In [3]:
response = swap.Definition(
    instrument_tag="IRS-EURIBOR-5Y",
    start_date="2020-01-28",
    tenor="5Y",
    legs=[
        swap.LegDefinition(
            direction=swap.Direction.PAID,
            notional_ccy="EUR",
            interest_payment_frequency=swap.Frequency.ANNUAL,
            interest_type=swap.InterestType.FIXED,
            interest_calculation_method="Dcb_30_360",
        ),
        swap.LegDefinition(
            direction=swap.Direction.RECEIVED,
            notional_ccy="EUR",
            interest_payment_frequency=swap.Frequency.SEMI_ANNUAL,
            index_name="EURIBOR",
            interest_type=swap.InterestType.FLOAT,
            interest_calculation_method="Dcb_Actual_360",
            index_tenor="6M",
            spread_bp=0.0,
        ),
    ],
    pricing_parameters=swap.PricingParameters(
        valuation_date="2020-01-26T00:00:00Z"
    ),
    fields=["InstrumentTag",
            "DirtyPricePercent",
            "FixedRatePercent",
            "SpreadBp",
            "PV01Bp",
            "PV01AmountInDealCcy",
            "DiscountCurveName",
            "ForwardCurveName",
            "ErrorCode",
            "ErrorMessage"],
).get_data()

response.data.df

,InstrumentTag,DirtyPricePercent,FixedRatePercent,SpreadBp,PV01Bp,PV01AmountInDealCcy,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,IRS-EURIBOR-5Y,-1.126908,-0.223562,<NA>,-0.034005,-3.40048,EUR - Swap vs 6M Euribor,<NA>,,
1,IRS-EURIBOR-5Y,-1.126908,<NA>,0,-4.569057,-456.905679,EUR - Swap vs 6M Euribor,EUR - Swap vs 6M Euribor,,


In [4]:
rd.close_session()